Libraries Being used
#Pandas, Numpy, yfinance, ta, scikit-learn, matplotlib, backtrader

In [2]:
# Data Handling
import pandas as pd
import numpy as np

# Financial Data
import yfinance as yf

# Technical Analysis
import ta

# Machine Learning
import sklearn

# Visualization
import matplotlib.pyplot as plt

# Backtesting
import backtrader as bt

In [35]:
import yfinance as yf

# Define the ticker symbol
ticker_symbol = "AAPL"

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Fetch historical market data
historical_data = ticker.history(period="1y")  # data for the last year
print("Historical Data:")
print(historical_data)

# Fetch basic financials
financials = ticker.financials
print("\nFinancials:")
print(financials)

# Fetch stock actions like dividends and splits
actions = ticker.actions
print("\nStock Actions:")
print(actions)

Historical Data:
                                 Open        High         Low       Close  \
Date                                                                        
2024-01-30 00:00:00-05:00  190.009132  190.864939  186.556047  187.123260   
2024-01-31 00:00:00-05:00  186.128151  186.187872  183.451278  183.501022   
2024-02-01 00:00:00-05:00  183.093032  186.038593  182.923863  185.949036   
2024-02-02 00:00:00-05:00  178.983166  186.416750  178.376139  184.943970   
2024-02-05 00:00:00-05:00  187.232740  188.327383  184.934004  186.765030   
...                               ...         ...         ...         ...   
2025-01-24 00:00:00-05:00  224.779999  225.630005  221.410004  222.779999   
2025-01-27 00:00:00-05:00  224.020004  232.149994  223.979996  229.860001   
2025-01-28 00:00:00-05:00  230.850006  240.190002  230.809998  238.259995   
2025-01-29 00:00:00-05:00  234.119995  239.860001  234.009995  239.360001   
2025-01-30 00:00:00-05:00  238.664993  240.789993  237.2100

Simple Model

In [39]:

# List of ticker symbols
ticker_symbols = ["MSFT", "AAPL", "GOOGL"]

# Loop through each ticker symbol and fetch data
for ticker_symbol in ticker_symbols:
    # Create a Ticker object for each stock
    ticker = yf.Ticker(ticker_symbol)

    # Fetch historical market data for the last 30 days
    historical_data = ticker.history(period="1mo")  # data for the last month

    # Add a new column 'Ticker' with the ticker symbol for each row
    historical_data['Ticker'] = ticker_symbol

    # Display a summary of the fetched data
    print(f"Summary of Historical Data for {ticker_symbol}:")
    print(historical_data[['Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']])
    print("\n" + "-"*50 + "\n")

Summary of Historical Data for MSFT:
                          Ticker        Open        High         Low  \
Date                                                                   
2024-12-30 00:00:00-05:00   MSFT  426.059998  427.549988  421.899994   
2024-12-31 00:00:00-05:00   MSFT  426.100006  426.730011  420.660004   
2025-01-02 00:00:00-05:00   MSFT  425.529999  426.070007  414.850006   
2025-01-03 00:00:00-05:00   MSFT  421.079987  424.029999  419.540009   
2025-01-06 00:00:00-05:00   MSFT  428.000000  434.320007  425.480011   
2025-01-07 00:00:00-05:00   MSFT  429.000000  430.649994  420.799988   
2025-01-08 00:00:00-05:00   MSFT  423.459991  426.970001  421.540009   
2025-01-10 00:00:00-05:00   MSFT  424.630005  424.709991  415.019989   
2025-01-13 00:00:00-05:00   MSFT  415.239990  418.500000  412.290009   
2025-01-14 00:00:00-05:00   MSFT  417.809998  419.739990  410.720001   
2025-01-15 00:00:00-05:00   MSFT  419.130005  428.149994  418.269989   
2025-01-16 00:00:00-05:00  

Top 100 Stocks over the past 5 years

In [44]:
import yfinance as yf
import pandas as pd

# List of the top 10 ticker symbols
ticker_symbols = [ "MSFT", "AAPL", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "SPY", "V", "AMD",
    "NVDA", "INTC", "CSCO", "WMT", "DIS", "MCD", "BA", "PYPL", "SNAP", "NKE",
    "XOM", "CVX", "JNJ", "PFE", "BABA", "T", "VZ", "GE", "IBM", "GS", "JPM",
    "MS", "GS", "UNH", "CVS", "HD", "LOW", "UPS", "SBUX", "BLK", "DELL", "LMT",
    "ABT", "TXN", "TMO", "MRK", "AMAT", "INTU", "ADI", "MU", "LULU", "PLUG",
    "F", "GM", "COF", "STZ", "KMX", "TSCO", "LMT", "SHOP", "SQ", "BIDU", "SPG",
    "MELI", "BA", "ATVI", "GS", "CAT", "HLT", "TGT", "DOW", "COP", "BA", "NKE",
    "LULU", "GS", "JNJ", "JPM", "BRK-A", "MSFT", "V", "TSLA", "XOM", "PFE",
    "WMT", "GE", "T", "VZ", "AMZN", "BABA", "CSCO", "NVDA", "AAPL", "GOOGL", "INTC",
    "AMT", "CVX", "META", "WFC", "KO", "PEP", "K", "SBUX", "NKE", "LOW", "MCD",
    "DIS", "WMT", "XOM", "UPS", "HD", "TMO", "PYPL", "UBER", "ZM", "TEAM", "MDB",
    "DKNG"]

# Create an empty DataFrame to store all the data
all_data = pd.DataFrame()

# Loop through each ticker symbol and fetch data
for ticker_symbol in ticker_symbols:
    # Create a Ticker object for each stock
    ticker = yf.Ticker(ticker_symbol)

    # Fetch historical market data from 2024 to the present
    historical_data = ticker.history(start="2020-01-01", end=pd.to_datetime("today").strftime('%Y-%m-%d'))

    # Add a new column 'Ticker' with the ticker symbol for each row
    historical_data['Ticker'] = ticker_symbol

    # Append the data to the all_data DataFrame
    all_data = pd.concat([all_data, historical_data])

# Reset the index after combining all the data
all_data.reset_index(inplace=True)

# Display the first few rows of the combined data
print(all_data[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']])

# Optionally, save the data to a CSV file
all_data.to_csv('top_10_stocks_2024.csv', index=False)
print("Data saved to top_10_stocks_2024.csv")

$ATVI: possibly delisted; no timezone found
C:\Users\Sezy\AppData\Local\Temp\ipykernel_33544\2875241900.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_data = pd.concat([all_data, historical_data])


       Ticker                      Date        Open        High         Low  \
0        MSFT 2020-01-02 00:00:00-05:00  151.870762  153.735906  151.440346   
1        MSFT 2020-01-03 00:00:00-05:00  151.430809  152.989871  151.182113   
2        MSFT 2020-01-06 00:00:00-05:00  150.244705  152.176809  149.699501   
3        MSFT 2020-01-07 00:00:00-05:00  152.387267  152.722028  150.474296   
4        MSFT 2020-01-08 00:00:00-05:00  152.014192  153.802829  151.076840   
...       ...                       ...         ...         ...         ...   
149287   DKNG 2025-01-23 00:00:00-05:00   40.770000   41.535000   40.250000   
149288   DKNG 2025-01-24 00:00:00-05:00   41.570000   42.000000   41.090000   
149289   DKNG 2025-01-27 00:00:00-05:00   40.500000   41.455002   40.070000   
149290   DKNG 2025-01-28 00:00:00-05:00   41.250000   42.279999   41.230000   
149291   DKNG 2025-01-29 00:00:00-05:00   42.200001   42.305000   41.200001   

             Close      Volume  
0       153.630692